## 1. 네이버 뉴스 데이터 가져오기

In [1]:
import os
import sys
import urllib.request
import requests

news_data = []
page_count = 3

client_id = "3kQhlGXxOc34pzRN58b8"
client_secret = "KlTURzIdS_"
encText = urllib.parse.quote("파이썬")

for idx in range(page_count):
    # json
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&start=" + str(idx * 10 + 1)

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):

        result = requests.get(response.geturl(),
                              headers={"X-Naver-Client-Id":client_id,
                                       "X-Naver-Client-Secret":client_secret}
                             )
        news_data.append(result.json())

    else:
        print("Error Code:" + rescode)

### 네이버 뉴스만 저장

In [2]:
naver_news_link = []

for page in news_data:
#     print(page)
    page_news_link = []
    
    for item in page['items']:
        #print(item)
        temp_link = item['link']
#         print(temp_link)
        if "naver" in temp_link:
            page_news_link.append(temp_link)
    
    naver_news_link.append(page_news_link)

### 네이버 뉴스 기사 크롤링

In [3]:
import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm import tqdm_notebook
import requests
import pickle
import re
import ast

from bs4 import BeautifulSoup 
from urllib.request import urlopen
import urllib
import time

In [4]:
driver = webdriver.Chrome('/Users/seonil/Desktop/driver/chromedriver')

In [5]:
naver_news_title = []
naver_news_content = []


for n in tqdm_notebook(range(len(naver_news_link))):
#     print(n)
    news_page_title = []
    news_page_content = []
    
    for idx in tqdm_notebook(range(len(naver_news_link[n]))):
        
        
    # URL로 접속
        try:
            driver.get(naver_news_link[n][idx])
            print(naver_news_link[n][idx])
            
        except:
            print("Timeout!")
            continue
        
        
        try:
            response = driver.page_source
            
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().accept()
            print("게시글이 삭제된 경우입니다.")
            continue
        
        soup = BeautifulSoup(response, "html.parser")
        
        # 뉴스 타이틀
        
        title = None
        
        try:
            item = soup.find('div', class_="article_info")
            title = item.find('h3', class_="tts_head").get_text()
            #print(title)

        except:
            title = "OUTLINK"
        
#         print(title)
        news_page_title.append(title)
        
        
        # 뉴스 본문
        
        doc = None
        text = ""
                
        data = soup.find_all("div", {"class" : "_article_body_contents"})
        if data:
            for item in data:

                text = text + str(item.find_all(text=True)).strip()
                text = ast.literal_eval(text)
                doc = ' '.join(text)
   
        else:
            doc = "OUTLINK"
            
        news_page_content.append(doc.replace('\n', ' '))

                
    naver_news_title.append(news_page_title)
    naver_news_content.append(news_page_content)

    time.sleep(2)

/Users/seonil/opt/anaconda3/envs/test/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/Users/seonil/opt/anaconda3/envs/test/lib/python3.6/site-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=421&aid=0005556704
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=008&aid=0004634761
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=009&aid=0004841708



https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010676924
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=015&aid=0004594713




### 크롤링 결과 저장

In [6]:
with open("naver_news_title.pk", "wb") as f:
    pickle.dump(naver_news_title, f)
    
with open("naver_news_content.pk", "wb") as f:
    pickle.dump(naver_news_content, f)

## 2. 토픽 모델링 (gensim)

In [7]:
from tqdm import tqdm_notebook 
from konlpy.tag import Mecab 
import numpy as np
import string 
import re
import warnings 
import pickle
from gensim import corpora
from gensim import models
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning) 

### 전처리 함수

In [8]:
def read_documents(input_file_name):
    
    corpus = []
    
    with open(input_file_name, 'rb') as f:
        temp_corpus = pickle.load(f)
        
    for page in temp_corpus:
        corpus += page
    
    return corpus

def text_cleaning(docs):
    
    for doc in docs:
        doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)

    return docs

def define_stopwords(path):
    
    SW = set()
    
    for i in string.punctuation:
        SW.add(i)
    
    with open(path) as f:
        for word in f:
            SW.add(word)

    return SW


def text_tokenizing(corpus, tokenizer):
    
    mecab = Mecab()
    token_corpus = []
    

    if tokenizer == "noun":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.nouns(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
                
            token_corpus.append(token_text)
            
    elif tokenized == "morph":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)

    elif tokenizer == "word":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
        

    return token_corpus


input_file_name = "data/naver_news_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("data/stopwords-ko.txt")
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun") #tokenizer= "noun" or "word"

In [9]:
# bag of words
dictionary = corpora.Dictionary(tokenized_text)
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

In [10]:
# lda model
model = models.ldamodel.LdaModel(corpus, num_topics=4, id2word=dictionary)

In [11]:
# 마지막 토픽, 10개 단어
model.show_topic(3, 10)

[('교육', 0.049934156),
 ('인공지능', 0.026237886),
 ('과정', 0.020828623),
 ('한밭', 0.015528885),
 ('데이터', 0.014485034),
 ('운영', 0.014448632),
 ('세종', 0.014016202),
 ('러닝', 0.012420734),
 ('공학', 0.012376833),
 ('실습', 0.011855137)]

### 토픽 모델링 함수

In [12]:
NUM_TOPICS = 3

NUM_TOPIC_WORDS = 30


def build_doc_term_mat(documents):
    print("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary


def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

In [13]:
corpus, dictionary = build_doc_term_mat(tokenized_text)

# LDA 모델
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)

Building document-term matrix.

Printing topic words.

Topic ID: 0
	교육	0.02070821076631546
	과정	0.014254316687583923
	인공지능	0.013551284559071064
	미적분	0.011044965125620365
	데이터	0.010591119527816772
	실습	0.01046698447316885
	프로그램	0.009771065786480904
	한밭	0.009417698718607426
	러닝	0.009303211234509945
	코딩	0.009276061318814754
	플레이어	0.00905511062592268
	내용	0.00903499685227871
	진행	0.00902926828712225
	대학	0.008764331229031086
	예측	0.008472765795886517
	교수	0.008428006432950497
	기술	0.008040238171815872
	이해	0.00790377426892519
	생각	0.007842546328902245
	재직	0.007841642946004868
	본문	0.007793664466589689
	플랫	0.007595655508339405
	운영	0.007143128663301468
	세종	0.006995231844484806
	분석	0.006823406089097261
	대학교	0.006680227350443602
	신경망	0.006381955463439226
	오류	0.00629609078168869
	공학	0.006060867570340633
	우회	0.006028519943356514


Topic ID: 1
	교육	0.04051082953810692
	인공지능	0.02221476472914219
	과정	0.01519868429750204
	한밭	0.012104225344955921
	데이터	0.011565291322767735
	운영	0.011323790065944195
	세종	0.0112582975